<a href="https://colab.research.google.com/github/sujitpal/nlp-deeplearning-ai-examples/blob/master/x2_building_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

## Exercise 2.01

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv

--2020-07-27 23:08:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11979363 (11M) [application/x-httpd-php]
Saving to: ‘energydata_complete.csv.1’

energydata_complete 100%[===================>]  11.42M  34.2MB/s    in 0.3s    

2020-07-27 23:08:33 (34.2 MB/s) - ‘energydata_complete.csv.1’ saved [11979363/11979363]



In [3]:
energy_df = pd.read_csv("energydata_complete.csv")
energy_df = energy_df.drop(columns=["date"])
energy_df.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


## Exercise 2.02

**Any Categorical Features?**
There are no non-numerical (categorical) columns.

In [4]:
cols = energy_df.columns
num_cols = energy_df._get_numeric_data().columns
list(set(cols) - set(num_cols))

[]

**Missing Values?**

In [5]:
energy_df.isnull().sum()

Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

**Outliers?**

In [6]:
outliers = {}
for col in energy_df.columns:
  col_values = energy_df[col].values
  col_mean, col_std = np.mean(col_values), np.std(col_values)
  col_lb, col_ub = col_mean - 3 * col_std, col_mean + 3 * col_std
  count = 0
  for x in col_values:
    if x < col_lb and x > col_ub:
      count += 1
  if count > 0:
    percentage = count / len(col_values)
    outliers[col] = "{:.3f}".format(percentage)

outliers

{}

## Exercise 2.03

**Rescaling**

In [7]:
X = energy_df.iloc[:, 1:]
y = energy_df.iloc[:, 0]

In [8]:
Xmin, Xmax = np.min(X, axis=0), np.max(X, axis=0)
ymin, ymax = np.min(y), np.max(y)

In [9]:
X = (X - Xmin) / (Xmax - Xmin)
y = (y - ymin) / (ymax - ymin)

In [10]:
X.head()

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.428571,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,0.381691,0.381070,0.841827,0.170594,0.653428,0.173329,0.661412,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.428571,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,0.381691,0.375443,0.839872,0.170594,0.651064,0.173329,0.660155,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.428571,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,0.380037,0.367487,0.830704,0.170594,0.646572,0.173329,0.655586,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.571429,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,0.380037,0.363800,0.833401,0.164310,0.641489,0.164175,0.650788,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.571429,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,0.380037,0.361859,0.848264,0.170594,0.639362,0.164175,0.650788,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


## Exercise 2.04

**Splitting dataset**

In [11]:
X.shape

(19735, 27)

In [12]:
train_end = int(X.shape[0] * 0.6)
dev_end = int(X.shape[0] * 0.8)

train_end, dev_end

(11841, 15788)

In [13]:
X_shuffle = X.sample(frac=1, random_state=0)
y_shuffle = y.sample(frac=1, random_state=0)


In [14]:
Xtrain = X_shuffle.iloc[0 : train_end, :].values
ytrain = y_shuffle.iloc[0 : train_end].values
Xdev = X_shuffle.iloc[train_end : dev_end, :].values
ydev = y_shuffle.iloc[train_end : dev_end].values
Xtest = X_shuffle.iloc[dev_end: , :].values
ytest = y_shuffle.iloc[dev_end:].values

Xtrain.shape, ytrain.shape, Xdev.shape, ydev.shape, Xtest.shape, ytest.shape

((11841, 27), (11841,), (3947, 27), (3947,), (3947, 27), (3947,))

In [15]:
Xtrain = torch.tensor(Xtrain, dtype=torch.float32).to("cuda")
ytrain = torch.tensor(ytrain, dtype=torch.float32).to("cuda")
Xdev = torch.tensor(Xdev, dtype=torch.float32).to("cuda")
ydev = torch.tensor(ydev, dtype=torch.float32).to("cuda")
Xtest = torch.tensor(Xtest, dtype=torch.float32).to("cuda")
ytest = torch.tensor(ytest, dtype=torch.float32).to("cuda")

## Exercise 2.05

**Building a Deep Neural Network**

In [16]:
model = nn.Sequential(
    nn.Linear(Xtrain.shape[1], 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 25),
    nn.ReLU(),
    nn.Linear(25, 1)
)
model = model.to("cuda")
model

Sequential(
  (0): Linear(in_features=27, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=25, bias=True)
  (5): ReLU()
  (6): Linear(in_features=25, out_features=1, bias=True)
)

In [17]:
loss_fn = torch.nn.MSELoss()

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [19]:
for i in range(1000):
  y_pred = model(Xtrain).squeeze()
  loss = loss_fn(y_pred, ytrain)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if i % 100 == 0:
    print(i, loss.item())

0 0.02776564285159111
100 0.008520383387804031
200 0.007633619010448456
300 0.007075233850628138
400 0.006632211618125439
500 0.006249370984733105
600 0.005939819384366274
700 0.005636805202811956
800 0.0053865546360611916
900 0.005197406746447086


In [20]:
pred = model(Xtest[0])
print("Ground truth: {:.3f}, prediction: {:.3f}".format(
    ytest[0].cpu().numpy(), 
    pred[0].cpu().detach().numpy()))

Ground truth: 0.047, prediction: 0.047
